In [435]:
import pandas as pd
from Reprezentacja_sumacyjna import CreateTable

In [436]:
from typing import List, Tuple

In [437]:
df = CreateTable("0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15")
df1 = df.get_df()
df_grupowane2 = pd.DataFrame()
df_grupowane2[["Obiekt grupowany", "Elemtny(łączone)", "Element"]] = df1[['Liczba jedynek', 'Liczba Dziesiętna', 'Liczba Binarna']]
df_grupowane2.info()

0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15 


In [413]:
df_groupwane = pd.DataFrame(
    columns=["Obiekt grupowany", "Elemtny(łączone)", "Element"],
    data=[[0, 0, '0000'],
            [1, 2, '0010'],
            [1, 8, '1000'],
            [2, 3, '0011'],
            [2, 5, '0101'],
            [2, 6, '0110'],
            [2, 9, '1001'],
            [2, 10, '1010'],
            [2, 12, '1100'],
            [3, 7, '0111'],
            [3, 11, '1011'],
            [3, 13, '1101'],
            [3, 14, '1110'],
            [4, 15, '1111']])
df_groupwane

,Obiekt grupowany,Elemtny(łączone),Element
0,0,0,0000
1,1,2,0010
2,1,8,1000
3,2,3,0011
4,2,5,0101
5,2,6,0110
6,2,9,1001
7,2,10,1010
8,2,12,1100
9,3,7,0111


In [414]:
def matched(num1: str, num2: str) -> bool:
    return sum(n1 != n2 for n1, n2 in zip(num1, num2)) == 1

In [415]:
assert matched('0000', '000x') == True

In [416]:
def correct_pair(num1: str, num2: str) -> str:
    return ''.join(n1 if n1 == n2 else '-' for n1, n2 in zip(num1, num2))

In [417]:
assert correct_pair('0010', '0000') == '00-0'

In [418]:
def correct_tuple(tuple_elements: Tuple) -> Tuple:
    tup_element = tuple_elements
    # print(f"tupl_elem, {tup_element}")
    while any(isinstance(el, tuple) for el in tup_element):
        tup_element = sum(tup_element, ())
    tup_element = tuple(set(tup_element))
    return tup_element

In [419]:
assert correct_tuple(((1, 2), (3, 4))) == (1, 2, 3, 4)

In [420]:
assert correct_tuple((1, 2, 3, 4)) == (1, 2, 3, 4)

In [421]:
def elementy_laczone(df_input: pd.DataFrame) -> list:
    return pd.Series(df_input["Elemtny(łączone)"].values, index=df_input['Element']).to_dict()

In [426]:
def remove_element(dict_: dict, element):
    if element in dict_.keys():
        dict_.pop(element)

In [427]:
from typing import Dict

In [428]:
def compare_neighbours(df_input: pd.DataFrame, main: int, next_: int, dict_lacz: Dict[str, 'int']={})\
        -> Tuple[pd.DataFrame, list]:
    # print(f"El_laczone = {el_lacz}")
    df_new = pd.DataFrame(columns=["Obiekt grupowany", "Elemtny(łączone)", "Element", "Użyty"])
    df_main = df_input.copy()
    # print each comparsion
    main_rows = df_main[df_main["Obiekt grupowany"] == main]
    next_rows = df_main[df_main["Obiekt grupowany"] == next_]
    # print(f"next->{next_rows}")

    for idx_main, row_main in main_rows.iterrows():
        for idx_, row_ in next_rows.iterrows():
            check1 = row_main['Element']
            check2 = row_['Element']
            if matched(check1, check2):
                # print(f"Matched! -{check1}- with {check2}")
                # print(f"nr -{row_main['Elemtny(łączone)']}- with {row_['Elemtny(łączone)']}")
                remove_element(dict_lacz, row_main["Element"])
                remove_element(dict_lacz, row_["Element"])
                if df_new.empty:
                    x = 0
                else:
                    x = df_new.index.max() + 1
                grupowy_tuple = correct_tuple(tuple([main]) + tuple([next_]))
                laczony_tuple = correct_tuple(tuple([row_main["Elemtny(łączone)"]]) + tuple([row_["Elemtny(łączone)"]]))
                df_new.loc[x] = [grupowy_tuple,
                                 laczony_tuple,
                                 correct_pair(check1, check2),
                                 False]
    return df_new, dict_lacz

In [429]:
def create_group(df_input: pd.DataFrame, dict_lacz: dict={}) -> pd.DataFrame:
    df_main = df_input.copy()
    df_out = pd.DataFrame()
    dict_lacz.update(elementy_laczone(df_input))
    unique_values = list(df_input['Obiekt grupowany'].unique())
    # print(f"Unique elements len = {len(unique_values)}")
    # print(f"Unique elements = {unique_values}")
    # print(f"Uprise group len = ->{len(unique_values) - 1}<-")
    for idx, el in enumerate(unique_values):
        if el == unique_values[-1]: continue
        df_tmp, dict_lacz = compare_neighbours(df_main, unique_values[idx], unique_values[idx+1], dict_lacz)
        df_out = pd.concat([df_out, df_tmp])
    df_out = df_out.drop_duplicates(subset='Element')
    return df_out.reset_index(drop=True), dict_lacz

In [466]:
def all_groups(df_input: pd.DataFrame):
    df_all: List[pd.DataFrame] = []
    df_out, dict_out = create_group(df_input, {})
    while not df_out.empty:
        df_out, dict_out = create_group(df_out, dict_out)
    print(f"List out: {dict_out}")
    return dict_out

In [467]:
def przygotuj_tab_pokryc(postac_sumacyjna: List[int], implikanty_proste: dict):
    print(f"Postac sumacyjna: {postac_sumacyjna}")
    print("Słownik")
    for key, item in implikanty_proste.items():
        print(f"key[{key}] = {item}")
    df_tab_pokryc = pd.DataFrame(
        columns=list(implikanty_proste.keys()),
        index=postac_sumacyjna,
        data=[['-'] * len(list(implikanty_proste.keys()))]
    )
    return df_tab_pokryc

In [468]:
def wypelnij_tab_pokryc(df_wstepna: pd.DataFrame, implikanty_proste: dict):
    for column, elements in implikanty_proste.items():
        # print(f"column = {column}")
        for element in elements:
            # print(f"element = {element}")
            df_wstepna.at[element, column] = '+'  # index, col
    return df_wstepna

In [486]:
df = CreateTable("0 2 3 5 6 7 8 9 10 11 12 13 14 15")
df1 = df.get_df()
df_grupowane2 = pd.DataFrame()
df_grupowane2[["Obiekt grupowany", "Elemtny(łączone)", "Element"]] = df1[['Liczba jedynek', 'Liczba Dziesiętna', 'Liczba Binarna']]
df_grupowane2

0 2 3 5 6 7 8 9 10 11 12 13 14 15 


,Obiekt grupowany,Elemtny(łączone),Element
0,0,0,0000
1,1,2,0010
2,1,8,1000
3,2,3,0011
4,2,5,0101
5,2,6,0110
6,2,9,1001
7,2,10,1010
8,2,12,1100
9,3,7,0111


In [490]:
df_grupowane2

,Obiekt grupowany,Elemtny(łączone),Element
0,0,0,0000
1,1,2,0010
2,1,8,1000
3,2,3,0011
4,2,5,0101
5,2,6,0110
6,2,9,1001
7,2,10,1010
8,2,12,1100
9,3,7,0111


In [491]:
compare_neighbours(df_grupowane2, 0, 1)

(  Obiekt grupowany Elemtny(łączone) Element  Użyty
 0           (0, 1)           (0, 2)    00-0  False
 1           (0, 1)           (0, 8)    -000  False,
 {})

In [487]:
dict_out = all_groups(df_grupowane2)
dict_out

List out: {'-0-0': (0, 8, 2, 10), '-1-1': (13, 15, 5, 7), '--1-': (2, 3, 6, 7, 10, 11, 14, 15), '1---': (8, 9, 10, 11, 12, 13, 14, 15)}


{'-0-0': (0, 8, 2, 10),
 '-1-1': (13, 15, 5, 7),
 '--1-': (2, 3, 6, 7, 10, 11, 14, 15),
 '1---': (8, 9, 10, 11, 12, 13, 14, 15)}

In [488]:
sorted(df.get_df()['Liczba Dziesiętna'])

[0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [489]:
postac_sumacyjna: List[int] = sorted(df.get_df()['Liczba Dziesiętna'])
out_ = przygotuj_tab_pokryc(postac_sumacyjna, dict_out)
wypelnij_tab_pokryc(out_, dict_out)

Postac sumacyjna: [0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Słownik
key[-0-0] = (0, 8, 2, 10)
key[-1-1] = (13, 15, 5, 7)
key[--1-] = (2, 3, 6, 7, 10, 11, 14, 15)
key[1---] = (8, 9, 10, 11, 12, 13, 14, 15)


,-0-0,-1-1,--1-,1---
0,+,-,-,-
2,+,-,+,-
3,-,-,+,-
5,-,+,-,-
6,-,-,+,-
7,-,+,+,-
8,+,-,-,+
9,-,-,-,+
10,+,-,+,+
11,-,-,+,+
